In [11]:
import requests
from bs4 import BeautifulSoup
import bs4
import json
import httpx
import time
import lxml
import cohere
from cohere.classify import Example 
import pandas as pd
from unidecode import unidecode

In [12]:
cohere_apiKey = '6rIWhaa4vGVQKRnSJ9ClSKkgsxQp8EabiXBLH2zl'
elsevier_apikey = 'ce04ffa20c7ee4560e4ffd4761bbe157'

In [13]:
class paper_parser():
    def __init__(self, query, apikey, count, start, sort, view, timeout):
        self.query = query
        self.apikey = apikey
        self.count = count
        self.start = start
        self.sort = sort
        self.view = view
        self.timeout = timeout
        self.dic = self.search_for_papers(self.query, self.apikey, self.count, self.start, self.sort, self.view)
        self.piis = list(self.dic.values())
        self.paragraphs = []
    def search_for_papers(self, query, apikey, count, start, sort, view):
        url = 'https://api.elsevier.com/content/search/sciencedirect?'
        headers = {'X-ELS-APIKey': apikey, 'Accept': 'application/json'}
        params = {'query': query, 'count': count, 'start': start, 'sort': sort, 'view': view}
        r = requests.get(url, headers=headers, params=params)
        json_resp = json.loads(r.text)
        titles, piis = [], []
        dic = {}
        for paper in json_resp['search-results']['entry']:
            titles.append(paper['dc:title'])
            piis.append(paper['pii'])
            dic[paper['dc:title']] = paper['pii']
        return dic
    def get_paragraphs(self, pii, apikey, timeout):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = timeout)
        soup = BeautifulSoup(response.content, "xml")
        text =[unidecode(''.join(s.findAll(text=True)).replace('\n', '')) for s in soup.find_all('ce:para')]
        return text
    def get_all_paragraphs(self):
        for pii in self.piis:
            self.paragraphs.append(self.get_paragraphs(pii, self.apikey, self.timeout))
        return self.paragraphs
    def get_all_titles(self):
        return list(self.dic.keys())
    def get_text(self, pii, apikey, timeout):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = timeout)
        soup = BeautifulSoup(response.content, "xml")
        return soup

        

In [14]:
pp = paper_parser('nips, membrane, polysulfone', elsevier_apikey, 10, 0, 'relevance', 'COMPLETE', 60)
example_papers = pp.get_all_paragraphs()
example_titles = pp.get_all_titles()

KeyError: 'search-results'

In [ ]:
pp.get_text(pp.piis[0], elsevier_apikey, 60)

<?xml version="1.0" encoding="utf-8"?>
<full-text-retrieval-response xmlns="http://www.elsevier.com/xml/svapi/article/dtd" xmlns:bk="http://www.elsevier.com/xml/bk/dtd" xmlns:cals="http://www.elsevier.com/xml/common/cals/dtd" xmlns:ce="http://www.elsevier.com/xml/common/dtd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:ja="http://www.elsevier.com/xml/ja/dtd" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:prism="http://prismstandard.org/namespaces/basic/2.0/" xmlns:sa="http://www.elsevier.com/xml/common/struct-aff/dtd" xmlns:sb="http://www.elsevier.com/xml/common/struct-bib/dtd" xmlns:tb="http://www.elsevier.com/xml/common/table/dtd" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xocs="http://www.elsevier.com/xml/xocs/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><coredata><prism:url>https://api.elsevier.com/content/article/pii/S037673882200343X</prism:url><dc:identifier>doi:10.1016/j.memsci.2022.120597</dc:identifier><

In [ ]:
example_papers[0]

['Versatile, facile, easy, and economical, these are the advantages that immersion precipitation or non-solvent phase separation (NIPS) offers [1-3]. Due to these, NIPS process has been regarded as the most widely-used approach in fabricating flat sheet membranes from a variety of polymers [4]. In addition, NIPS can provide a wide-range of pore size: from microfiltration to nanofiltration [5]. In this process, a polymer of choice is dissolved in a suitable solvent. Once homogeneous, the polymeric solution is cast onto a support and phase-inversion is prompted through direct immersion in a coagulation bath/non-solvent bath (typically water), generating a polymer rich phase that transforms into the membrane matrix after polymer precipitation, and a polymer lean phase which develops into pores after demixing [6].',
 'Polysulfone (PSf) membrane is amongst the most extensively used commercial membranes due to its favorable attributes such as high glass transition temperature (Tg = 190 degC)

In [ ]:
example_titles

['Strategy to prepare skin-free and macrovoid-free polysulfone membranes via the NIPS process',
 'Towards high-performance polysulfone membranes: A controllable membrane formation process using surfactant in NIPS',
 'Efficient dye rejection using a mixed matrix polyphenylsulfone/polysulfone membrane containing basil seed mucilage hydrogel',
 'Tight UF membranes with ultrahigh water flux prepared by <ce:italic>in-situ</ce:italic> growing ZIF particles in NIPS process for greatly enhanced dye removal efficiency',
 'Preparation of mixed matrix membranes made up of polysulfone and MIL-53(Al) nanoparticles as promising membranes for separation of aqueous dye solutions',
 'Fabrication of polysulfone membrane with sponge-like structure by using different non-woven fabrics',
 'Photocatalytic polysulfone membrane incorporated by ZnO-MnO<ce:inf\xa0loc=post>2</ce:inf>@SiO<ce:inf\xa0loc=post>2</ce:inf> composite under UV light irradiation for the reliable treatment of natural rubber-laden wastewat

In [75]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [76]:
Example_data = pd.DataFrame(flatten_list(example_papers), columns = ['paragraph'])
Example_data.to_csv('Example_data.csv')

In [59]:
co = cohere.Client(cohere_apiKey) 
response = co.classify( 
  model='large', 
  inputs=["Am I still able to return my order?", "When can I expect my package?"], 
  examples=[Example("Do you offer same day shipping?", "Shipping and handling policy"), Example("Can you ship to Italy?", "Shipping and handling policy"), Example("How long does shipping take?", "Shipping and handling policy"), Example("Can I buy online and pick up in store?", "Shipping and handling policy"), Example("What are your shipping options?", "Shipping and handling policy"), Example("My order arrived damaged, can I get a refund?", "Start return or exchange"), Example("You sent me the wrong item", "Start return or exchange"), Example("I want to exchange my item for another colour", "Start return or exchange"), Example("I ordered something and it wasn’t what I expected. Can I return it?", "Start return or exchange"), Example("What’s your return policy?", "Start return or exchange"), Example("Where’s my package?", "Track order"), Example("When will my order arrive?", "Track order"), Example("What’s my shipping number?", "Track order"), Example("Which carrier is my package with?", "Track order"), Example("Is my package delayed?", "Track order")]) 
print('The confidence levels of the labels are: {}'.format(response.classifications)) 

The confidence levels of the labels are: [Classification<prediction: "Start return or exchange", confidence: 0.9938701>, Classification<prediction: "Track order", confidence: 0.995854>]
